#  **Step 1**. Cloning the LPRNet Model

In [ ]:
!git clone https://github.com/sirius-ai/LPRNet_Pytorch.git

fatal: destination path 'LPRNet_Pytorch' already exists and is not an empty directory.


In [ ]:
%cd LPRNet_Pytorch

/content/LPRNet_Pytorch


In [ ]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import time
import os
import numpy as np
from sklearn.cluster import KMeans
import subprocess
import re
import matplotlib.pyplot as plt
from data.load_data import CHARS, CHARS_DICT, LPRDataLoader
from PIL import Image, ImageDraw, ImageFont
from model.LPRNet import build_lprnet
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import *
from torch import optim
import torch.nn as nn
import numpy as np
import argparse
import torch
import time
import cv2
import os

## **1.1** Loading test dataset

In [ ]:
# Loading the test dataset
test_dataset = LPRDataLoader(img_dir='/content/LPRNet_Pytorch/data/test',
                             imgSize=(94, 24),
                             lpr_max_len=8)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
import os

def count_images_in_dir(img_dir):
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif')  # Add extensions as needed
    count = 0

    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if file.lower().endswith(valid_extensions):
                count += 1

    return count

# Specify the directory
img_dir = '/content/LPRNet_Pytorch/data/test'

# Count and print the number of images
image_count = count_images_in_dir(img_dir)
print(f"Number of images in directory '{img_dir}': {image_count}")

Number of images in directory '/content/LPRNet_Pytorch/data/test': 1000


In [ ]:
from data.load_data import CHARS, CHARS_DICT, LPRDataLoader
from PIL import Image, ImageDraw, ImageFont
from model.LPRNet import build_lprnet
# import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import *
from torch import optim
import torch.nn as nn
import numpy as np
import argparse
import torch
import time
import cv2
import os

In [ ]:
CHARS = ['京', '沪', '津', '渝', '冀', '晋', '蒙', '辽', '吉', '黑',
         '苏', '浙', '皖', '闽', '赣', '鲁', '豫', '鄂', '湘', '粤',
         '桂', '琼', '川', '贵', '云', '藏', '陕', '甘', '青', '宁',
         '新',
         '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
         'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K',
         'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
         'W', 'X', 'Y', 'Z', 'I', 'O', '-'
         ]
def get_parser():
    class Args:
        img_size = [94, 24]
        test_img_dirs = "./data/test"
        dropout_rate = 0.5
        lpr_max_len = 8
        test_batch_size = 100
        phase_train = False
        num_workers = 8
        cuda = False
        show = False
        pretrained_model = './weights/Final_LPRNet_model.pth'
    return Args()

def collate_fn(batch):
    imgs = []
    labels = []
    lengths = []
    for _, sample in enumerate(batch):
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        labels.extend(label)
        lengths.append(length)
    labels = np.asarray(labels).flatten().astype(np.float32)

    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)

def test(model_path):
    args = get_parser()
    args.pretrained_model = model_path

    lprnet = build_lprnet(lpr_max_len=args.lpr_max_len, phase=args.phase_train, class_num=len(CHARS), dropout_rate=args.dropout_rate)
    device = torch.device("cuda:0" if args.cuda else "cpu")
    lprnet.to(device)
    print("Successful to build network!")

    # load pretrained model
    if args.pretrained_model:
        checkpoint = torch.load(args.pretrained_model, map_location=torch.device('cpu'))
        lprnet.load_state_dict(checkpoint) # Load state dict to the model
        print("load pretrained model successful!")
    else:
        print("[Error] Can't found pretrained mode, please check!")
        return False

    test_img_dirs = os.path.expanduser(args.test_img_dirs)
    test_dataset = LPRDataLoader(test_img_dirs, imgSize=(94, 24), lpr_max_len=8) # Remove split(',')
    try:
        Greedy_Decode_Eval(lprnet, test_dataset, args)
    finally:
      pass


def Greedy_Decode_Eval(Net, datasets, args):
    # TestNet = Net.eval()
    epoch_size = len(datasets) // args.test_batch_size
    batch_iterator = iter(DataLoader(datasets, args.test_batch_size, shuffle=True, num_workers=args.num_workers, collate_fn=collate_fn))

    Tp = 0
    Tn_1 = 0
    Tn_2 = 0
    t1 = time.time()
    for i in range(epoch_size):
        # load train data
        images, labels, lengths = next(batch_iterator)
        start = 0
        targets = []
        for length in lengths:
            label = labels[start:start+length]
            targets.append(label)
            start += length
        targets = np.array([el.numpy() for el in targets])
        imgs = images.numpy().copy()

        if args.cuda:
            images = Variable(images.cuda())
        else:
            images = Variable(images)

        # forward
        prebs = Net(images)
        # greedy decode
        prebs = prebs.cpu().detach().numpy()
        preb_labels = list()
        for i in range(prebs.shape[0]):
            preb = prebs[i, :, :]
            preb_label = list()
            for j in range(preb.shape[1]):
                preb_label.append(np.argmax(preb[:, j], axis=0))
            no_repeat_blank_label = list()
            pre_c = preb_label[0]
            if pre_c != len(CHARS) - 1:
                no_repeat_blank_label.append(pre_c)
            for c in preb_label: # dropout repeate label and blank label
                if (pre_c == c) or (c == len(CHARS) - 1):
                    if c == len(CHARS) - 1:
                        pre_c = c
                    continue
                no_repeat_blank_label.append(c)
                pre_c = c
            preb_labels.append(no_repeat_blank_label)
        for i, label in enumerate(preb_labels):
            # show image and its predict label
            if args.show:
                show(imgs[i], label, targets[i])
            if len(label) != len(targets[i]):
                Tn_1 += 1
                continue
            if (np.asarray(targets[i]) == np.asarray(label)).all():
                Tp += 1
            else:
                Tn_2 += 1
    Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)
    print("[Info] Test Accuracy: {} [{}:{}:{}:{}]".format(Acc, Tp, Tn_1, Tn_2, (Tp+Tn_1+Tn_2)))
    t2 = time.time()
    print("[Info] Test Speed: {}s 1/{}]".format((t2 - t1) / len(datasets), len(datasets)))

def show(img, label, target):
  pass

def cv2ImgAddText(img, text, pos, textColor=(255, 0, 0), textSize=12):
    if (isinstance(img, np.ndarray)):  # detect opencv format or not
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    fontText = ImageFont.truetype("data/NotoSansCJK-Regular.ttc", textSize, encoding="utf-8")
    draw.text(pos, text, textColor, font=fontText)

    return cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2BGR)



In [ ]:
from model.LPRNet import LPRNet
def load_pretrained_model(checkpoint_path):
    lprnet = LPRNet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
    checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    if 'state_dict' in checkpoint:
        lprnet.load_state_dict(checkpoint['state_dict'])
    elif 'lprnet' in checkpoint:
        lprnet.load_state_dict(checkpoint['lprnet'])
    else:
        lprnet.load_state_dict(checkpoint)
    lprnet.eval()
    return lprnet

In [ ]:
# Path to the pretrained model checkpoint
checkpoint_path = './weights/Final_LPRNet_model.pth'

# Load the original model
original_model = load_pretrained_model(checkpoint_path)

input_shape = (1, 3, 24, 94)  # Adjust based on your input size

<ipython-input-9-0b66248fb090>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))


In [ ]:
import torch
from model.LPRNet import build_lprnet

# Load pretrained model
model = build_lprnet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
model.load_state_dict(torch.load('./weights/Final_LPRNet_model.pth', map_location = torch.device('cpu')))
model.eval()

# Export to ONNX
dummy_input = torch.randn(1, 3, 24, 94)  # Input size for LPRNet
torch.onnx.export(model, dummy_input, "./weights/Original_LPRNet.onnx", verbose=True)
print("Model exported to ONNX format.")

<ipython-input-11-0f599f939cf6>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./weights/Final_LPRNet_model.pth', map_location = torch.de

Model exported to ONNX format.


In [ ]:
# Check ONNX file size
import os
onnx_file = "./weights/Original_LPRNet.onnx"
size_mb = os.path.getsize(onnx_file) / (1024 * 1024)
print(f"ONNX Model Size: {size_mb:.2f} MB")


ONNX Model Size: 1.91 MB


In [ ]:
import onnx
import numpy as np

# Load the ONNX model
onnx_file = "./weights/Original_LPRNet.onnx"
onnx_model = onnx.load(onnx_file)

# Data type size lookup table (in bytes)
dtype_size_map = {
    onnx.TensorProto.FLOAT: 4,    # 32-bit float
    onnx.TensorProto.INT64: 8,   # 64-bit integer
    onnx.TensorProto.INT32: 4,   # 32-bit integer
    onnx.TensorProto.INT16: 2,   # 16-bit integer
    onnx.TensorProto.INT8: 1,    # 8-bit integer
    onnx.TensorProto.UINT8: 1    # 8-bit unsigned integer
}

# Calculate non-zero size
non_zero_size_bytes = 0
for initializer in onnx_model.graph.initializer:
    dtype = initializer.data_type
    dtype_size = dtype_size_map.get(dtype, 0)  # Get the size of each data type
    weights = np.frombuffer(initializer.raw_data, dtype=np.float32)  # Assume float32 by default
    non_zero_count = np.count_nonzero(weights)
    non_zero_size_bytes += non_zero_count * dtype_size

# Convert size to MB
non_zero_size_mb = non_zero_size_bytes / (1024 * 1024)
print(f"ONNX Model Size (Non-Zero Parameters Only): {non_zero_size_mb:.2f} MB")


ONNX Model Size (Non-Zero Parameters Only): 1.11 MB


In [ ]:
inference_times=[]
test_accuracy=[]


In [ ]:
!python test_LPRNet.py --test_img_dirs 'data/test' --pretrained_model 'weights/Final_LPRNet_model.pth'

Traceback (most recent call last):
  File "/content/LPRNet_Pytorch/test_LPRNet.py", line 176, in <module>
    test()
  File "/content/LPRNet_Pytorch/test_LPRNet.py", line 60, in test
    lprnet.to(device)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1340, in to
    return self._apply(convert)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 900, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 927, in _apply
    param_applied = fn(param)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1326, in convert
    return t.to(
  File "/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py", line 319, in _lazy_init
    torch._C._cuda_init()
RuntimeError: Found no NVIDIA driver on your system. Please check

# **Step 2.** MODEL OPTIMIZATION 1: Unstructured Pruning

In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import time
import copy

In [ ]:
def get_model_size(model):
    num_nonzeros = sum(param.count_nonzero().item() for param in model.parameters())
    print(f"No. of Non-Zero Parameters: {num_nonzeros}")
    return num_nonzeros

def unstructured_pruning(model, prune_ratios):
    pruned_models = []
    for prune_ratio in prune_ratios:
        pruned_model = copy.deepcopy(model)
        for module in pruned_model.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                prune.l1_unstructured(module, name="weight", amount=prune_ratio)
        pruned_models.append(pruned_model)
    return pruned_models

def measure_inference_time(model, input_shape, num_iterations=100):
    device = next(model.parameters()).device
    dummy_input = torch.randn(input_shape).to(device)


    with torch.no_grad():
        for _ in range(10):
            _ = model(dummy_input)

    # Measuring inference time
    start_time = time.time()
    with torch.no_grad():
        for _ in range(num_iterations):
            _ = model(dummy_input)
    end_time = time.time()

    avg_inference_time = (end_time - start_time) / num_iterations
    return avg_inference_time * 1000  # Convert to milliseconds

In [ ]:
# Loading the original model
original_model = load_pretrained_model(checkpoint_path)
print("LPRNet model loaded successfully!")
print("Original model:")
original_size = get_model_size(original_model)
input_shape = (1, 3, 24, 94)

# Measuring the original inference time
original_inference_time = measure_inference_time(original_model, input_shape)
print(f"Original Inference Time: {original_inference_time:.4f} ms")

prune_ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] ## pruning ratios
# Apply unstructured pruning to the original model
pruned_models = unstructured_pruning(original_model.cpu(), prune_ratios)



<ipython-input-9-0b66248fb090>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))


LPRNet model loaded successfully!
Original model:
No. of Non-Zero Parameters: 446975
Original Inference Time: 217.3214 ms


In [ ]:
for i, pruned_model in enumerate(pruned_models):
    # Remove pruning reparameterization
    for module in pruned_model.modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.remove(module, "weight")

    # Get pruned model size
    print(f"\nPruned model with ratio {prune_ratios[i]}:")
    pruned_size = get_model_size(pruned_model)

    # Calculate and print the reduction percentage
    reduction_percentage = (original_size - pruned_size) / original_size * 100
    print(f"Parameter reduction: {reduction_percentage:.2f}%")

    # Measure inference time
    inference_time = measure_inference_time(pruned_model, input_shape)
    print(f"Inference Time: {inference_time:.4f} ms")

    # Save the pruned model
    if prune_ratios[i]==0.5:
      model_path = f"unstructured_pruned_LPRNet_model_{prune_ratios[i]}.pth"
      torch.save(pruned_model.state_dict(), model_path)
    else:
      model_path = f"unstructured_pruned_LPRNet_model.pth"
      torch.save(pruned_model.state_dict(), model_path)

    # Test the pruned model
    print(f"Testing model with pruning ratio: {prune_ratios[i]}")
    test(model_path)
    print("--------------------")


Pruned model with ratio 0.1:
No. of Non-Zero Parameters: 402641
Parameter reduction: 9.92%
Inference Time: 217.4282 ms
Testing model with pruning ratio: 0.1
Successful to build network!
load pretrained model successful!


<ipython-input-8-bd08ba42b3bf>:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.pretrained_model, map_location=torch.device('cpu'))
/usr/local/

[Info] Test Accuracy: 0.899 [899:64:37:1000]
[Info] Test Speed: 0.18981312584877014s 1/1000]
--------------------

Pruned model with ratio 0.2:
No. of Non-Zero Parameters: 358305
Parameter reduction: 19.84%
Inference Time: 181.7911 ms
Testing model with pruning ratio: 0.2
Successful to build network!
load pretrained model successful!
[Info] Test Accuracy: 0.9 [900:61:39:1000]
[Info] Test Speed: 0.18516061782836915s 1/1000]
--------------------

Pruned model with ratio 0.3:
No. of Non-Zero Parameters: 313975
Parameter reduction: 29.76%
Inference Time: 153.4372 ms
Testing model with pruning ratio: 0.3
Successful to build network!
load pretrained model successful!
[Info] Test Accuracy: 0.878 [878:72:50:1000]
[Info] Test Speed: 0.1677018654346466s 1/1000]
--------------------

Pruned model with ratio 0.4:
No. of Non-Zero Parameters: 269639
Parameter reduction: 39.67%
Inference Time: 136.9080 ms
Testing model with pruning ratio: 0.4
Successful to build network!
load pretrained model success

#### **2.1** Selecting pruning ratio

From the above pruning ratios, unstructured pruning has given best results on pruning ratio 0.5. Beyond 0.5, accuracy starts descending. On pruning ratio 0.5, the number of paramters have reduced by approximately 50%, therefore achieving the balance of achieving a decent accuracy along with having less parameters.

In [ ]:
!pip install onnx

In [ ]:
import torch
from model.LPRNet import build_lprnet

# Load pretrained model
model = build_lprnet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
model.load_state_dict(torch.load('/content/LPRNet_Pytorch/unstructured_pruned_LPRNet_model_0.5.pth', map_location = torch.device('cpu')))
model.eval()

# Export to ONNX
dummy_input = torch.randn(1, 3, 24, 94)  # Input size for LPRNet
torch.onnx.export(model, dummy_input, "./weights/UnstructurePrune_LPRNet.onnx", verbose=True)
print("Model exported to ONNX format.")

<ipython-input-19-6d2d64de2025>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/LPRNet_Pytorch/unstructured_pruned_LPRNet_model_0

Model exported to ONNX format.


#### **2.2** Reduced Model Size

In [ ]:
import onnx
import numpy as np

# Load the ONNX model
onnx_file = "./weights/UnstructurePrune_LPRNet.onnx"
onnx_model = onnx.load(onnx_file)

# Data type size lookup table (in bytes)
dtype_size_map = {
    onnx.TensorProto.FLOAT: 4,    # 32-bit float
    onnx.TensorProto.INT64: 8,   # 64-bit integer
    onnx.TensorProto.INT32: 4,   # 32-bit integer
    onnx.TensorProto.INT16: 2,   # 16-bit integer
    onnx.TensorProto.INT8: 1,    # 8-bit integer
    onnx.TensorProto.UINT8: 1    # 8-bit unsigned integer
}

# Calculate non-zero size
non_zero_size_bytes = 0
for initializer in onnx_model.graph.initializer:
    dtype = initializer.data_type
    dtype_size = dtype_size_map.get(dtype, 0)  # Get the size of each data type
    weights = np.frombuffer(initializer.raw_data, dtype=np.float32)  # Assume float32 by default
    non_zero_count = np.count_nonzero(weights)
    non_zero_size_bytes += non_zero_count * dtype_size

# Convert size to MB
non_zero_size_mb = non_zero_size_bytes / (1024 * 1024)
print(f"ONNX Model Size (Non-Zero Parameters Only): {non_zero_size_mb:.2f} MB")


ONNX Model Size (Non-Zero Parameters Only): 0.71 MB


#### **2.3** Optimization Summary

**Unstructured Pruned Model Summary**
(Pruning Ratio: 0.5)

Test Accuracy:0.823

---


Test Speed:0.143

---



Number of non zero parameters of Pruned Model: 225304

---


Parameter Reduction: 49.59%

---

ONNX Model Size (Non-Zero Parameters Only): 0.71 MB



# **Step 3.** MLC

### **3.1** MLC & Model Conversions

In [ ]:
!python3 -m  pip install mlc-ai-cpu -f https://mlc.ai/wheels

Looking in links: https://mlc.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.8/185.8 MB 3.9 MB/s eta 0:00:00


In [ ]:
import torch
import tvm
from tvm import relay
import numpy as np
from tvm.contrib import graph_executor, relay_viz

In [ ]:
lprnet = build_lprnet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
lprnet.load_state_dict(torch.load('/content/LPRNet_Pytorch/weights/Final_LPRNet_model.pth', map_location = torch.device('cpu')))
lprnet.eval()


<ipython-input-57-0596785d3c69>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load('/content/LPRNet_Pytorch/weights/Final_LPRNet_model.pth', 

LPRNet(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (4): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (3): ReLU()
        (4): Conv2d(32, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (5): ReLU()
        (6): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2), padding=0, dilation=1, ceil_mode=False)
    (8): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 64, 

In [ ]:
# Defining input
input_shape = (1, 3, 24, 94)
input_data = torch.randn(input_shape)

In [ ]:
# Converting the PyTorch Model to its TorchScript Module
torchScriptModel = torch.jit.trace(lprnet, input_data)

In [ ]:
# Defining input shapes for TVM:
input_name = "tvmData"
shapeList = [(input_name, input_shape)]

In [ ]:
# Converting the TorchScript model to TVM's Relay intermediate representation.
mod, params = relay.frontend.from_pytorch(torchScriptModel, shapeList)

# Compiling the Relay module into a deployable library
target = "llvm"
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)
    print(lib)


In [ ]:
dev = tvm.cpu(0)
tvmGModule = graph_executor.GraphModule(lib["default"](dev))

In [ ]:
inputData_nmpy = np.random.rand(1, 3, 24, 94).astype("float32")
tvmGModule.set_input("tvmData", inputData_nmpy)


tvmGModule.run()


output = tvmGModule.get_output(0).asnumpy()
print("Output Shape:", output.shape)

Output Shape: (1, 68, 18)


In [ ]:
# Visualizing the Relay Model
viz = relay_viz.RelayVisualizer(mod)
viz.render()

@main([Var(tvmData, ty=TensorType([1, 3, 24, 94], float32)), Var(aten::_convolution_0.weight, ty=TensorType([64, 3, 3, 3], float32)), Var(aten::_convolution_0.bias, ty=TensorType([64], float32)), Var(aten::batch_norm_0.weight, ty=TensorType([64], float32)), Var(aten::batch_norm_0.bias, ty=TensorType([64], float32)), Var(aten::batch_norm_0.running_mean, ty=TensorType([64], float32)), Var(aten::batch_norm_0.running_var, ty=TensorType([64], float32)), Var(aten::_convolution_1.weight, ty=TensorType([32, 64, 1, 1], float32)), Var(aten::_convolution_1.bias, ty=TensorType([32], float32)), Var(aten::_convolution_2.weight, ty=TensorType([32, 32, 3, 1], float32)), Var(aten::_convolution_2.bias, ty=TensorType([32], float32)), Var(aten::_convolution_3.weight, ty=TensorType([32, 32, 1, 3], float32)), Var(aten::_convolution_3.bias, ty=TensorType([32], float32)), Var(aten::_convolution_4.weight, ty=TensorType([128, 32, 1, 1], float32)), Var(aten::_convolution_4.bias, ty=TensorType([128], float32)), V

## **3.2** Autotuning

In [ ]:
import torch
import tvm
from tvm import relay, autotvm, auto_scheduler

In [ ]:
lprnet = build_lprnet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
lprnet.load_state_dict(torch.load('/content/LPRNet_Pytorch/weights/Final_LPRNet_model.pth', map_location = torch.device('cpu')))
lprnet.eval()

<ipython-input-66-0596785d3c69>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load('/content/LPRNet_Pytorch/weights/Final_LPRNet_model.pth', 

LPRNet(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (4): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (3): ReLU()
        (4): Conv2d(32, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (5): ReLU()
        (6): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2), padding=0, dilation=1, ceil_mode=False)
    (8): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 64, 

In [ ]:
# Defining input
inputShape = (1, 3, 24, 94)
inputData = torch.randn(inputShape)

In [ ]:
# Defining input shapes for TVM
inputName = "tvmData"
shapeList = [(inputName, inputShape)]

In [ ]:
# Convert model to a TorchScript module
torchScriptModel = torch.jit.trace(lprnet, inputData)

# Convert PyTorch model to Relay:
mod, params = relay.frontend.from_pytorch(torchScriptModel, shapeList)

In [ ]:
# Extracting the scheduling tasks from the Relay model and printing their computation DAGs
# Compile to TensorIR :
target = "llvm"
taskLogFile = "mlctuningLogs.json"

print("Extract following tasks...")
tasks, task_weights = auto_scheduler.extract_tasks(mod["main"], params, target)

for idx, task in enumerate(tasks):
    print("========== Task Number - %d  (workload key: %s):" % (idx, task.workload_key))
    print(task.compute_dag)


Extract following tasks...
========== Task Number - 0  (workload key: ["82c9323a6bf75f7be33236a5c2cf52cb", [1, 1, 128, 20, 90], [1, 1, 64, 18, 44]]):
p0 = PLACEHOLDER [1, 1, 128, 20, 90]
pool_max(ax0, ax1, ax2, ax3, ax4) max= p0[ax0, ax1, ((ax2*2) + rv0), (ax3 + rv1), ((ax4*2) + rv2)]

========== Task Number - 1  (workload key: ["ddfc1ebe8806378ca6f78e8a1f6be501", [1, 16, 20, 90, 4], [8, 16, 1, 1, 4, 4], [1, 8, 1, 1, 4], [1, 8, 20, 90, 4]]):
p0 = PLACEHOLDER [1, 16, 20, 90, 4]
p1 = PLACEHOLDER [8, 16, 1, 1, 4, 4]
conv2d_NCHWc(n, oc_chunk, oh, ow, oc_block) += (p0[n, floordiv(ic, 4), (oh + kh), (ow + kw), floormod(ic, 4)]*p1[oc_chunk, floordiv(ic, 4), kh, kw, floormod(ic, 4), oc_block])
p2 = PLACEHOLDER [1, 8, 1, 1, 4]
T_add(ax0, ax1, ax2, ax3, ax4) = (conv2d_NCHWc[ax0, ax1, ax2, ax3, ax4] + p2[ax0, ax1, 0, 0, ax4])
T_relu(ax0, ax1, ax2, ax3, ax4) = max(T_add[ax0, ax1, ax2, ax3, ax4], 0f)

========== Task Number - 2  (workload key: ["58bc26dbe1c1ee728a7795a35ebc9164", [1, 64, 18, 44, 4]

In [ ]:
# Task Scheduling and Tuning with TVM Auto-Scheduler
tuner = auto_scheduler.TaskScheduler(tasks, task_weights)
tune_option = auto_scheduler.TuningOptions(
    num_measure_trials=150,
    runner=auto_scheduler.LocalRunner(repeat=10, enable_cpu_cache_flush=True),
    measure_callbacks=[auto_scheduler.RecordToFile(taskLogFile)],
)

tuner.tune(tune_option)

|  ID  |                       Task Description                        | Latency (ms) | Speed (GFLOPS) | Trials |
-----------------------------------------------------------------------------------------------------------------
|    0 |                                  vm_mod_fused_nn_max_pool3d_1 |            - |              - |      0 |
|    1 |              vm_mod_fused_nn_contrib_conv2d_NCHWc_add_nn_relu |            - |              - |      0 |
|    2 |                                  vm_mod_fused_nn_avg_pool2d_2 |            - |              - |      0 |
|    3 |            vm_mod_fused_nn_contrib_conv2d_NCHWc_add_nn_relu_5 |            - |              - |      0 |
|    4 |        vm_mod_fused_nn_contrib_conv2d_NCHWc_add_add_nn_relu_1 |            - |              - |      0 |
|    5 |            vm_mod_fused_nn_contrib_conv2d_NCHWc_add_nn_relu_1 |            - |              - |      0 |
|    6 |                                           vm_mod_fused_mean_4 |            - | 

In [ ]:
import numpy as np
# Compile with the history best
print("Compile...")
with auto_scheduler.ApplyHistoryBest(taskLogFile):
    with tvm.transform.PassContext(opt_level=3, config={"relay.backend.use_auto_scheduler": True}):
        lib = relay.build(mod, target=target, params=params)

Compile...


In [ ]:
# Create graph executor
dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))
data_tvm = tvm.nd.array((np.random.uniform(size=input_shape)).astype("float32"))
module.set_input("tvmData", data_tvm)


In [ ]:
# Evaluate
print("Evaluate inference time cost...")
print(module.benchmark(dev, repeat=3, min_repeat_ms=500))


Evaluate inference time cost...
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  36.2389      35.9469      36.9507      35.8192       0.5060                  


In [ ]:
import time
from data.load_data import CHARS, CHARS_DICT, LPRDataLoader
from PIL import Image, ImageDraw, ImageFont
from model.LPRNet import build_lprnet
# import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import *
from torch import optim
import torch.nn as nn
import numpy as np
import argparse
import torch
import time
import cv2
import os

In [ ]:
def collating_function(batch):
    imgs = []
    labels = []
    lengths = []
    for _, sample in enumerate(batch):
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        labels.extend(label)
        lengths.append(length)
    labels = np.asarray(labels).flatten().astype(np.float32)

    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)

In [ ]:
def tvm_test(model_type):
  test_img_dirs = os.path.expanduser('/content/LPRNet_Pytorch/data/test')
  datasets = LPRDataLoader(test_img_dirs, [94, 24], 8)

  epoch_size = len(datasets) // 1
  batch_iterator = iter(DataLoader(datasets, 1, shuffle=True, num_workers=8, collate_fn=collating_function))
  Tp = 0
  Tn_1 = 0
  Tn_2 = 0
  t1 = time.time()
  for i in range(epoch_size):
      # load train data
      images, labels, lengths = next(batch_iterator)
      start = 0
      targets = []
      for length in lengths:
          label = labels[start:start+length]
          targets.append(label)
          start += length
      targets = np.array([el.numpy() for el in targets])
      imgs = images.numpy().copy()

      images = Variable(images)
      if model_type == 'pytorch':
        prebs = quantized_model(images)
        prebs = prebs.cpu().detach().numpy()
      else:
        module.set_input("tvmData", tvm.nd.array(images.numpy()))


        module.run()

        # Getting output
        prebs = module.get_output(0).asnumpy()
      # decoding predictions
      preb_labels = list()
      for i in range(prebs.shape[0]):
          preb = prebs[i, :, :]
          preb_label = list()
          for j in range(preb.shape[1]):
              preb_label.append(np.argmax(preb[:, j], axis=0))
          no_repeat_blank_label = list()
          pre_c = preb_label[0]
          if pre_c != len(CHARS) - 1:
              no_repeat_blank_label.append(pre_c)
          for c in preb_label: # dropout repeate label and blank label
              if (pre_c == c) or (c == len(CHARS) - 1):
                  if c == len(CHARS) - 1:
                      pre_c = c
                  continue
              no_repeat_blank_label.append(c)
              pre_c = c
          preb_labels.append(no_repeat_blank_label)
      for i, label in enumerate(preb_labels):
          if len(label) != len(targets[i]):
              Tn_1 += 1
              continue
          if (np.asarray(targets[i]) == np.asarray(label)).all():
              Tp += 1
          else:
              Tn_2 += 1

  t2 = time.time()
  print("Test Speed: {}s 1/{}]".format((t2 - t1) / len(datasets), len(datasets)))
  Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)
  print("Test Accuracy: {} [{}:{}:{}:{}]".format(Acc, Tp, Tn_1, Tn_2, (Tp+Tn_1+Tn_2)))

## **3.3** Test Speed & Test Accuracy

In [ ]:
tvm_test(model_type= "tvm")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Test Speed: 0.04780561685562134s 1/1000]
Test Accuracy: 0.894 [894:69:37:1000]


## **3.4** Optimization Summary

**MLC Summary**


Test Accuracy:89%

---


Test Speed:0.047seconds

---

ONNX Model Size (Non-Zero Parameters Only): 1.11 MB



# **Step 4.** MLC on Pruned Model

## **4.1** MLC & Model Conversions

Passing pruned model for MLC optimization

In [ ]:
!python3 -m  pip install mlc-ai-cpu -f https://mlc.ai/wheels

Looking in links: https://mlc.ai/wheels


In [ ]:
import torch
import tvm
from tvm import relay
import numpy as np
from tvm.contrib import graph_executor, relay_viz

In [ ]:
lprnet = build_lprnet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
lprnet.load_state_dict(torch.load('/content/LPRNet_Pytorch/unstructured_pruned_LPRNet_model_0.5.pth', map_location = torch.device('cpu')))
lprnet.eval()


<ipython-input-24-ffc14f760cec>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load('/content/LPRNet_Pytorch/unstructured_pruned_LPRNet_model_

LPRNet(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (4): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (3): ReLU()
        (4): Conv2d(32, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (5): ReLU()
        (6): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2), padding=0, dilation=1, ceil_mode=False)
    (8): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 64, 

In [ ]:
# Defining input
input_shape = (1, 3, 24, 94)
input_data = torch.randn(input_shape)

In [ ]:
# Converting the PyTorch Model to its TorchScript Module
torchScriptModel = torch.jit.trace(lprnet, input_data)

In [ ]:
# Defining input shapes for TVM:
input_name = "tvmData"
shapeList = [(input_name, input_shape)]

In [ ]:
# Converting the TorchScript model to TVM's Relay intermediate representation.
mod, params = relay.frontend.from_pytorch(torchScriptModel, shapeList)

# Compiling the Relay module into a deployable library
target = "llvm"
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)
    print(lib)


In [ ]:
dev = tvm.cpu(0)
tvmGModule = graph_executor.GraphModule(lib["default"](dev))

In [ ]:
inputData_nmpy = np.random.rand(1, 3, 24, 94).astype("float32")
tvmGModule.set_input("tvmData", inputData_nmpy)


tvmGModule.run()


output = tvmGModule.get_output(0).asnumpy()
print("Output Shape:", output.shape)

Output Shape: (1, 68, 18)


In [ ]:
# Visualizing the Relay Model
viz = relay_viz.RelayVisualizer(mod)
viz.render()

@main([Var(tvmData, ty=TensorType([1, 3, 24, 94], float32)), Var(aten::_convolution_0.weight, ty=TensorType([64, 3, 3, 3], float32)), Var(aten::_convolution_0.bias, ty=TensorType([64], float32)), Var(aten::batch_norm_0.weight, ty=TensorType([64], float32)), Var(aten::batch_norm_0.bias, ty=TensorType([64], float32)), Var(aten::batch_norm_0.running_mean, ty=TensorType([64], float32)), Var(aten::batch_norm_0.running_var, ty=TensorType([64], float32)), Var(aten::_convolution_1.weight, ty=TensorType([32, 64, 1, 1], float32)), Var(aten::_convolution_1.bias, ty=TensorType([32], float32)), Var(aten::_convolution_2.weight, ty=TensorType([32, 32, 3, 1], float32)), Var(aten::_convolution_2.bias, ty=TensorType([32], float32)), Var(aten::_convolution_3.weight, ty=TensorType([32, 32, 1, 3], float32)), Var(aten::_convolution_3.bias, ty=TensorType([32], float32)), Var(aten::_convolution_4.weight, ty=TensorType([128, 32, 1, 1], float32)), Var(aten::_convolution_4.bias, ty=TensorType([128], float32)), V

## **4.2** Autotuning

In [ ]:
import torch
import tvm
from tvm import relay, autotvm, auto_scheduler

In [ ]:
lprnet = build_lprnet(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
lprnet.load_state_dict(torch.load('/content/LPRNet_Pytorch/unstructured_pruned_LPRNet_model_0.5.pth', map_location = torch.device('cpu')))
lprnet.eval()

<ipython-input-51-ffc14f760cec>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load('/content/LPRNet_Pytorch/unstructured_pruned_LPRNet_model_

LPRNet(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (4): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (3): ReLU()
        (4): Conv2d(32, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (5): ReLU()
        (6): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2), padding=0, dilation=1, ceil_mode=False)
    (8): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 64, 

In [ ]:
# Defining input
inputShape = (1, 3, 24, 94)
inputData = torch.randn(inputShape)

In [ ]:
# Defining input shapes for TVM
inputName = "tvmData"
shapeList = [(inputName, inputShape)]

In [ ]:
# Convert model to a TorchScript module
torchScriptModel = torch.jit.trace(lprnet, inputData)

# Convert PyTorch model to Relay:
mod, params = relay.frontend.from_pytorch(torchScriptModel, shapeList)

In [ ]:
# Extracting the scheduling tasks from the Relay model and printing their computation DAGs
# Compile to TensorIR :
target = "llvm"
taskLogFile = "tuningLogs.json"

print("Extract following tasks...")
tasks, task_weights = auto_scheduler.extract_tasks(mod["main"], params, target)

for idx, task in enumerate(tasks):
    print("========== Task Number - %d  (workload key: %s):" % (idx, task.workload_key))
    print(task.compute_dag)


Extract following tasks...
========== Task Number - 0  (workload key: ["82c9323a6bf75f7be33236a5c2cf52cb", [1, 1, 128, 20, 90], [1, 1, 64, 18, 44]]):
p0 = PLACEHOLDER [1, 1, 128, 20, 90]
pool_max(ax0, ax1, ax2, ax3, ax4) max= p0[ax0, ax1, ((ax2*2) + rv0), (ax3 + rv1), ((ax4*2) + rv2)]

========== Task Number - 1  (workload key: ["ddfc1ebe8806378ca6f78e8a1f6be501", [1, 16, 20, 90, 4], [8, 16, 1, 1, 4, 4], [1, 8, 1, 1, 4], [1, 8, 20, 90, 4]]):
p0 = PLACEHOLDER [1, 16, 20, 90, 4]
p1 = PLACEHOLDER [8, 16, 1, 1, 4, 4]
conv2d_NCHWc(n, oc_chunk, oh, ow, oc_block) += (p0[n, floordiv(ic, 4), (oh + kh), (ow + kw), floormod(ic, 4)]*p1[oc_chunk, floordiv(ic, 4), kh, kw, floormod(ic, 4), oc_block])
p2 = PLACEHOLDER [1, 8, 1, 1, 4]
T_add(ax0, ax1, ax2, ax3, ax4) = (conv2d_NCHWc[ax0, ax1, ax2, ax3, ax4] + p2[ax0, ax1, 0, 0, ax4])
T_relu(ax0, ax1, ax2, ax3, ax4) = max(T_add[ax0, ax1, ax2, ax3, ax4], 0f)

========== Task Number - 2  (workload key: ["58bc26dbe1c1ee728a7795a35ebc9164", [1, 64, 18, 44, 4]

In [ ]:
# Task Scheduling and Tuning with TVM Auto-Scheduler
tuner = auto_scheduler.TaskScheduler(tasks, task_weights)
tune_option = auto_scheduler.TuningOptions(
    num_measure_trials=150,
    runner=auto_scheduler.LocalRunner(repeat=10, enable_cpu_cache_flush=True),
    measure_callbacks=[auto_scheduler.RecordToFile(taskLogFile)],
)

tuner.tune(tune_option)

|  ID  |                       Task Description                        | Latency (ms) | Speed (GFLOPS) | Trials |
-----------------------------------------------------------------------------------------------------------------
|    0 |                                  vm_mod_fused_nn_max_pool3d_1 |            - |              - |      0 |
|    1 |              vm_mod_fused_nn_contrib_conv2d_NCHWc_add_nn_relu |            - |              - |      0 |
|    2 |                                  vm_mod_fused_nn_avg_pool2d_2 |            - |              - |      0 |
|    3 |            vm_mod_fused_nn_contrib_conv2d_NCHWc_add_nn_relu_5 |            - |              - |      0 |
|    4 |        vm_mod_fused_nn_contrib_conv2d_NCHWc_add_add_nn_relu_1 |            - |              - |      0 |
|    5 |            vm_mod_fused_nn_contrib_conv2d_NCHWc_add_nn_relu_1 |            - |              - |      0 |
|    6 |                                           vm_mod_fused_mean_4 |            - | 

In [ ]:
import numpy as np
# Compile with the history best
print("Compile...")
with auto_scheduler.ApplyHistoryBest(taskLogFile):
    with tvm.transform.PassContext(opt_level=3, config={"relay.backend.use_auto_scheduler": True}):
        lib = relay.build(mod, target=target, params=params)

Compile...


In [ ]:
# Create graph executor
dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))
data_tvm = tvm.nd.array((np.random.uniform(size=input_shape)).astype("float32"))
module.set_input("tvmData", data_tvm)


In [ ]:
# Evaluate
print("Evaluate inference time cost...")
print(module.benchmark(dev, repeat=3, min_repeat_ms=500))


Evaluate inference time cost...
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  34.4642      29.7384      46.3711      27.2831       8.4789                  


In [ ]:
import time
from data.load_data import CHARS, CHARS_DICT, LPRDataLoader
from PIL import Image, ImageDraw, ImageFont
from model.LPRNet import build_lprnet
# import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import *
from torch import optim
import torch.nn as nn
import numpy as np
import argparse
import torch
import time
import cv2
import os

In [ ]:
def collating_function(batch):
    imgs = []
    labels = []
    lengths = []
    for _, sample in enumerate(batch):
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        labels.extend(label)
        lengths.append(length)
    labels = np.asarray(labels).flatten().astype(np.float32)

    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)

In [ ]:
def tvm_test(model_type):
  test_img_dirs = os.path.expanduser('/content/LPRNet_Pytorch/data/test')
  datasets = LPRDataLoader(test_img_dirs, [94, 24], 8)

  epoch_size = len(datasets) // 1
  batch_iterator = iter(DataLoader(datasets, 1, shuffle=True, num_workers=8, collate_fn=collating_function))
  Tp = 0
  Tn_1 = 0
  Tn_2 = 0
  t1 = time.time()
  for i in range(epoch_size):
      # load train data
      images, labels, lengths = next(batch_iterator)
      start = 0
      targets = []
      for length in lengths:
          label = labels[start:start+length]
          targets.append(label)
          start += length
      targets = np.array([el.numpy() for el in targets])
      imgs = images.numpy().copy()

      images = Variable(images)
      if model_type == 'pytorch':
        prebs = quantized_model(images)
        prebs = prebs.cpu().detach().numpy()
      else:
        module.set_input("tvmData", tvm.nd.array(images.numpy()))


        module.run()

        # Getting output
        prebs = module.get_output(0).asnumpy()
      # decoding predictions
      preb_labels = list()
      for i in range(prebs.shape[0]):
          preb = prebs[i, :, :]
          preb_label = list()
          for j in range(preb.shape[1]):
              preb_label.append(np.argmax(preb[:, j], axis=0))
          no_repeat_blank_label = list()
          pre_c = preb_label[0]
          if pre_c != len(CHARS) - 1:
              no_repeat_blank_label.append(pre_c)
          for c in preb_label: # dropout repeate label and blank label
              if (pre_c == c) or (c == len(CHARS) - 1):
                  if c == len(CHARS) - 1:
                      pre_c = c
                  continue
              no_repeat_blank_label.append(c)
              pre_c = c
          preb_labels.append(no_repeat_blank_label)
      for i, label in enumerate(preb_labels):
          if len(label) != len(targets[i]):
              Tn_1 += 1
              continue
          if (np.asarray(targets[i]) == np.asarray(label)).all():
              Tp += 1
          else:
              Tn_2 += 1

  t2 = time.time()
  print("Test Speed: {}s 1/{}]".format((t2 - t1) / len(datasets), len(datasets)))
  Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)
  print("Test Accuracy: {} [{}:{}:{}:{}]".format(Acc, Tp, Tn_1, Tn_2, (Tp+Tn_1+Tn_2)))

## **4.3** Test Speed & Test Accuracy

In [ ]:
tvm_test(model_type= "tvm")

Test Speed: 0.039403223037719724s 1/1000]
Test Accuracy: 0.8 [800:123:77:1000]


## **4.4** Optimization Summary

**Pruned Model + MLC Summary**
(Pruning Ratio: 0.5)

Test Accuracy:80%

---


Test Speed:0.039 seconds

---

ONNX Model Size (Non-Zero Parameters Only): 0.71 MB



# **Step 5.** MODEL OPTIMIZATION 2:Quantization

The quantization method used in this code is post-training quantization, specifically static quantization with the fbgemm backend. In post-training quantization, the model is initially trained in floating-point precision, and quantization is applied afterward. Static quantization involves converting the model's weights and activations to a lower precision, such as int8, using a representative dataset for calibration before finalizing the conversion. The fbgemm backend, optimized for CPUs, enables efficient quantized operations and is commonly used for deep learning models running on CPU architectures. This approach helps reduce the model's size and boosts inference speed without requiring retraining.

In [ ]:
import torch.nn as nn
import torch


In [ ]:
class basicBlock(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(basicBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(ch_in, ch_out // 4, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out, kernel_size=1),
        )
    def forward(self, x):
        return self.block(x)

In [ ]:
class LPRNet(nn.Module):
    def __init__(self, lpr_max_len, phase, class_num, dropout_rate):
        super(LPRNet, self).__init__()
        self.quant = QuantStub()
        self.dequant = DeQuantStub()
        self.phase = phase
        self.lpr_max_len = lpr_max_len
        self.class_num = class_num
        self.backbone = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1), # 0
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),  # 2
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1)),
            basicBlock(ch_in=64, ch_out=128),    # *** 4 ***
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),  # 6
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2)),
            basicBlock(ch_in=64, ch_out=256),   # 8
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),  # 10
            basicBlock(ch_in=256, ch_out=256),   # *** 11 ***
            nn.BatchNorm2d(num_features=256),   # 12
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(4, 1, 2)),  # 14
            nn.Dropout(dropout_rate),
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=(1, 4), stride=1),  # 16
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),  # 18
            nn.Dropout(dropout_rate),
            nn.Conv2d(in_channels=256, out_channels=class_num, kernel_size=(13, 1), stride=1), # 20
            nn.BatchNorm2d(num_features=class_num),
            nn.ReLU(),  # *** 22 ***
        )
        self.container = nn.Sequential(
            nn.Conv2d(in_channels=448+self.class_num, out_channels=self.class_num, kernel_size=(1, 1), stride=(1, 1)),
        )

    def forward(self, x):
        keep_features = list()
        for i, layer in enumerate(self.backbone.children()):
            if isinstance(layer, nn.MaxPool3d):
                # print('max-pooling layer before')
                # print(x.shape)
                x = x.unsqueeze(0)
                x = layer(x)
                x = x.squeeze(0)
                # print('max-pooling layer after')
                # print(x.shape)
            else:
              x = layer(x)
            # print(layer, x.shape)
            if i in [2, 6, 13, 22]: # [2, 4, 8, 11, 22]
                keep_features.append(x)

        global_context = list()
        for i, f in enumerate(keep_features):
            if i in [0, 1]:
                f = nn.AvgPool2d(kernel_size=5, stride=5)(f)
            if i in [2]:
                f = nn.AvgPool2d(kernel_size=(4, 10), stride=(4, 2))(f)

            f = self.dequant(f)
            f_pow = torch.pow(f, 2)
            f_mean = torch.mean(f_pow)
            f = torch.div(f, f_mean)
            f = self.quant(f)
            global_context.append(f)

        x = torch.cat(global_context, 1)
        x = self.container(x)
        logits = torch.mean(x, dim=2)

        return logits

def build_lprnet_quantize(lpr_max_len=8, phase=False, class_num=66, dropout_rate=0.5):

    Net = LPRNet(lpr_max_len, phase, class_num, dropout_rate)

    if phase == "train":
        return Net.train()
    else:
        return Net.eval()

In [ ]:
qt = torch.randn(1,3,24, 94)

In [ ]:
print(qt.shape, qt.unsqueeze(2).shape)
layer = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2))
x= layer(qt)
print(x.shape)
x= layer(qt.unsqueeze(2))
print(x.shape)

torch.Size([1, 3, 24, 94]) torch.Size([1, 3, 1, 24, 94])
torch.Size([1, 2, 22, 46])
torch.Size([1, 3, 1, 22, 46])


In [ ]:
!pip install onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.8/725.8 kB 8.3 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.quantization
from torch.ao.quantization import QuantStub, DeQuantStub

In [ ]:
# Define a wrapper to add QuantStub and DeQuantStub
class QuantizedLPRNet(nn.Module):
    def __init__(self, model):
        super(QuantizedLPRNet, self).__init__()
        self.quant = QuantStub()
        self.model = model
        self.dequant = DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.model(x)
        x = self.dequant(x)
        return x

def load_pretrained_model(model_path):
    # Load the pretrained LPRNet model
    model = build_lprnet_quantize(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()
    return model

def apply_quantization(model, calibration_data):
    # Wrapping the model with QuantStub and DeQuantStub
    model = QuantizedLPRNet(model)

    # Define the quantization configuration
    model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

    # Prepare the model for quantization
    model_prepared = torch.ao.quantization.prepare(model, inplace = False)

    # Calibrate the model using a representative dataset
    model_prepared.eval()
    with torch.no_grad():
      model_prepared(calibration_data)

    # Convert to quantized version
    model_quantized = torch.ao.quantization.convert(model_prepared, inplace = False)
    return model_quantized

# # Load your trained model
model_path = 'weights/Final_LPRNet_model.pth'
model = load_pretrained_model(model_path)

calibration_data = torch.randn(1, 3, 24, 94) # Dummy data

# # Apply quantization
quantized_model = apply_quantization(model, calibration_data)

# # Save the quantized model
torch.save(quantized_model.state_dict(), 'Quantized_LPRNet_model.pth')




<ipython-input-33-3bc057ac8440>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
/usr/local/

In [ ]:
import time
from data.load_data import CHARS, CHARS_DICT, LPRDataLoader
from PIL import Image, ImageDraw, ImageFont
# from model.LPRNet import build_lprnet
# import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import *
from torch import optim
import torch.nn as nn
import numpy as np
import argparse
import torch
import time
import cv2
import os

Leveraging post-training static quantization to reduce the model size and improve inference performance without retraining, utilizing the fbgemm backend optimized for CPU-based inference.

#### **5.1** Test Speed & Test Accuracy

In [ ]:
def collating_function(batch):
    imgs = []
    labels = []
    lengths = []
    for _, sample in enumerate(batch):
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        labels.extend(label)
        lengths.append(length)
    labels = np.asarray(labels).flatten().astype(np.float32)

    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)

test_img_dirs = os.path.expanduser('/content/LPRNet_Pytorch/data/test')
datasets = LPRDataLoader(test_img_dirs, [94, 24], 8)

epoch_size = len(datasets) // 100
batch_iterator = iter(DataLoader(datasets, 100, shuffle=True, num_workers=8, collate_fn=collating_function))

#calibration:
model_path = 'weights/Final_LPRNet_model.pth'
new_model = load_pretrained_model(model_path)
calibration_data,_,_ = next(iter(DataLoader(datasets,100, shuffle = True, collate_fn = collate_fn)))
quantized_model = apply_quantization(new_model, calibration_data)


Tp = 0
Tn_1 = 0
Tn_2 = 0
t1 = time.time()
for i in range(epoch_size):
    # load train data
    images, labels, lengths = next(batch_iterator)
    start = 0
    targets = []
    for length in lengths:
        label = labels[start:start+length]
        targets.append(label)
        start += length
    targets = np.array([el.numpy() for el in targets])
    imgs = images.numpy().copy()

    images = Variable(images)


    prebs = quantized_model(images)
    # greedy decode
    prebs = prebs.cpu().detach().numpy()
    preb_labels = list()
    for i in range(prebs.shape[0]):
        preb = prebs[i, :, :]
        preb_label = list()
        for j in range(preb.shape[1]):
            preb_label.append(np.argmax(preb[:, j], axis=0))
        no_repeat_blank_label = list()
        pre_c = preb_label[0]
        if pre_c != len(CHARS) - 1:
            no_repeat_blank_label.append(pre_c)
        for c in preb_label: # dropout repeate label and blank label
            if (pre_c == c) or (c == len(CHARS) - 1):
                if c == len(CHARS) - 1:
                    pre_c = c
                continue
            no_repeat_blank_label.append(c)
            pre_c = c
        preb_labels.append(no_repeat_blank_label)
    for i, label in enumerate(preb_labels):
        if len(label) != len(targets[i]):
            Tn_1 += 1
            continue
        if (np.asarray(targets[i]) == np.asarray(label)).all():
            Tp += 1
        else:
            Tn_2 += 1
Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)
print("Test Accuracy: {} [{}:{}:{}:{}]".format(Acc, Tp, Tn_1, Tn_2, (Tp+Tn_1+Tn_2)))
t2 = time.time()
print("Test Speed: {}s 1/{}]".format((t2 - t1) / len(datasets), len(datasets)))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-33-3bc057ac8440>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary 

Test Accuracy: 0.743 [743:165:92:1000]
Test Speed: 0.028002031803131103s 1/1000]


In [ ]:
torch.save(quantized_model.state_dict(), 'Quantized_LPRNet_model.pth')

In [ ]:
import os

# Path to the saved quantized model
model_path = 'Quantized_LPRNet_model.pth'

# Get the size of the saved model file in bytes
model_size = os.path.getsize(model_path)

# Convert the size to MB for better readability
model_size_MB = model_size / (1024 * 1024)

print(f"The size of the quantized model is: {model_size_MB:.2f} MB")


The size of the quantized model is: 0.51 MB


In [ ]:
import torch
from model.LPRNet import build_lprnet

### **5.2** Optimization Summary

**Quantization Summary**

Test Accuracy:74.3%

---

Test Speed:0.028seconds

---

Model Size : 0.51 MB

# **Step 6.** MODEL OPTIMIZATION 2':Applying Quantization on Pruned Model

In [ ]:
# Define a wrapper to add QuantStub and DeQuantStub
class QuantizedLPRNet(nn.Module):
    def __init__(self, model):
        super(QuantizedLPRNet, self).__init__()
        self.quant = QuantStub()
        self.model = model
        self.dequant = DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.model(x)
        x = self.dequant(x)
        return x

def load_pretrained_model(model_path):
    # Load the pretrained LPRNet model
    model = build_lprnet_quantize(lpr_max_len=8, phase=False, class_num=68, dropout_rate=0.5)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()
    return model

def apply_quantization(model, calibration_data):
    # Wrap the model with QuantStub and DeQuantStub
    model = QuantizedLPRNet(model)

    # Define the quantization configuration
    model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

    # Prepare the model for quantization
    model_prepared = torch.ao.quantization.prepare(model, inplace = False)

    # Calibrate the model using a representative dataset
    model_prepared.eval()
    with torch.no_grad():
      model_prepared(calibration_data)

    # Convert to quantized version
    model_quantized = torch.ao.quantization.convert(model_prepared, inplace = False)
    return model_quantized

# # Load your trained model
model_path = '/content/LPRNet_Pytorch/unstructured_pruned_LPRNet_model_0.5.pth'
model = load_pretrained_model(model_path)


calibration_data = torch.randn(1, 3, 24, 94)

# # Apply quantization
quantized_model = apply_quantization(model, calibration_data)

# # Save the quantized model
torch.save(quantized_model.state_dict(), 'PruneQuantized_LPRNet_model.pth')




<ipython-input-39-b72ae64dcf04>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


## **6.1** Test Speed & Test Accuracy

In [ ]:
def collating_function(batch):
    imgs = []
    labels = []
    lengths = []
    for _, sample in enumerate(batch):
        img, label, length = sample
        imgs.append(torch.from_numpy(img))
        labels.extend(label)
        lengths.append(length)
    labels = np.asarray(labels).flatten().astype(np.float32)

    return (torch.stack(imgs, 0), torch.from_numpy(labels), lengths)

test_img_dirs = os.path.expanduser('/content/LPRNet_Pytorch/data/test')
datasets = LPRDataLoader(test_img_dirs, [94, 24], 8)

epoch_size = len(datasets) // 100
batch_iterator = iter(DataLoader(datasets, 100, shuffle=True, num_workers=8, collate_fn=collating_function))

#calibration:
model_path = '/content/LPRNet_Pytorch/unstructured_pruned_LPRNet_model_0.5.pth'
new_model = load_pretrained_model(model_path)
calibration_data,_,_ = next(iter(DataLoader(datasets,100, shuffle = True, collate_fn = collate_fn)))
quantized_model = apply_quantization(new_model, calibration_data)


Tp = 0
Tn_1 = 0
Tn_2 = 0
t1 = time.time()
for i in range(epoch_size):
    # load train data
    images, labels, lengths = next(batch_iterator)
    start = 0
    targets = []
    for length in lengths:
        label = labels[start:start+length]
        targets.append(label)
        start += length
    targets = np.array([el.numpy() for el in targets])
    imgs = images.numpy().copy()

    images = Variable(images)

    prebs = quantized_model(images)
    # greedy decode
    prebs = prebs.cpu().detach().numpy()
    preb_labels = list()
    for i in range(prebs.shape[0]):
        preb = prebs[i, :, :]
        preb_label = list()
        for j in range(preb.shape[1]):
            preb_label.append(np.argmax(preb[:, j], axis=0))
        no_repeat_blank_label = list()
        pre_c = preb_label[0]
        if pre_c != len(CHARS) - 1:
            no_repeat_blank_label.append(pre_c)
        for c in preb_label: # dropout repeate label and blank label
            if (pre_c == c) or (c == len(CHARS) - 1):
                if c == len(CHARS) - 1:
                    pre_c = c
                continue
            no_repeat_blank_label.append(c)
            pre_c = c
        preb_labels.append(no_repeat_blank_label)
    for i, label in enumerate(preb_labels):
        if len(label) != len(targets[i]):
            Tn_1 += 1
            continue
        if (np.asarray(targets[i]) == np.asarray(label)).all():
            Tp += 1
        else:
            Tn_2 += 1
Acc = Tp * 1.0 / (Tp + Tn_1 + Tn_2)
print("Test Accuracy: {} [{}:{}:{}:{}]".format(Acc, Tp, Tn_1, Tn_2, (Tp+Tn_1+Tn_2)))
t2 = time.time()
print("Test Speed: {}s 1/{}]".format((t2 - t1) / len(datasets), len(datasets)))

<ipython-input-39-b72ae64dcf04>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


Test Accuracy: 0.668 [668:204:128:1000]
Test Speed: 0.024468955993652345s 1/1000]


In [ ]:
torch.save(quantized_model.state_dict(), 'PrunedQuantized_LPRNet_model.pth')

In [ ]:
import os

# Path to the saved quantized model
model_path = 'PrunedQuantized_LPRNet_model.pth'

# Get the size of the saved model file in bytes
model_size = os.path.getsize(model_path)

# Convert the size to MB for better readability
model_size_MB = model_size / (1024 * 1024)

print(f"The size of the pruned and then quantized model is: {model_size_MB:.2f} MB")


The size of the pruned and then quantized model is: 0.51 MB


## **6.2** Optimization Summary

**Quantization on Pruned Model Summary**

Test Accuracy:66.8%

---

Test Speed:0.024 seconds

---

Model Size : 0.51 MB

# **Step 7.** Overall Summary

In [ ]:
import pandas as pd

columns = ['Model', 'Accuracy (%)', 'Inference Time (s)', 'Size (MB)']

data = [
    ['Orignial Model',90,0.23, '1.11(onnx)'],
    ['Pruned Model', 82.3, 0.143, '0.71 (onnx)'],
    ['MLC', 89.4,0.047, '1.11 (onnx)'],
    ['Pruned + MLC Optimized', 80, 0.039, '0.71 (onnx)'],
    ['Quantized Model', 74.3, 0.028, 0.51],
    ['Pruned+Quantized Model',66.8,0.0024, 0.51]]

df = pd.DataFrame(data, columns=columns)
print(df)


                    Model  Accuracy (%)  Inference Time (s)    Size (MB)
0          Orignial Model          90.0              0.2300   1.11(onnx)
1            Pruned Model          82.3              0.1430  0.71 (onnx)
2                     MLC          89.4              0.0470  1.11 (onnx)
3  Pruned + MLC Optimized          80.0              0.0390  0.71 (onnx)
4         Quantized Model          74.3              0.0280         0.51
5  Pruned+Quantized Model          66.8              0.0024         0.51
